In [1]:
import os
import numpy as np
import torch.nn.functional as F
import torch

from simba.model import Simba
from simba.functions import identify_baselines, matlab_baselines
from simba.util import print_all_perf, eval_simba, fix_seed, save_results

from simba.parameters import base_parameters, baselines_to_use
parameters = base_parameters

In [2]:
# Parameters
seed = 1
parameters['init_from_matlab_or_ls'] = True
parameters['max_epochs'] = 10000
parameters['init_epochs'] = 150000
parameters['print_each'] = 1000

# Simulation
dt = 1228.8
path_to_matlab = parameters['path_to_matlab']
directory = os.path.join('saves', f'Daisy_init_new_{seed}')
fix_seed(seed)

In [7]:
# Load and process data as in 
# https://homes.esat.kuleuven.be/~smc/daisy/daisydata.html

data = np.genfromtxt('data/powerplant.dat')
U = data[:,1:6]
Y = data[:,6:9]
Yr = data[:,9:12]

nu = U.shape[1]
ny = Y.shape[1]
H = Y.shape[0]

U = U.reshape(-1, H, nu)
Y = Y.reshape(-1, H, ny)

# Normalize
um = np.mean(U, axis=1, keepdims=True)
us = np.std(U, axis=1, keepdims=True)
U = (U - um) / us

ym = np.mean(Y, axis=1, keepdims=True)
ys = np.std(Y, axis=1, keepdims=True)
Y = (Y - ym) / ys

# Define everything
X = X_val = X_test = None
U_val = U[:,:150,:].copy()
Y_val = Y[:,:150,:].copy()
U_test = U[:,150:,:].copy()
Y_test = Y[:,150:,:].copy()
U = U[:,:100,:]
Y = Y[:,:100,:]

print(U.shape, Y.shape, U_val.shape, Y_val.shape, U_test.shape)

(1, 100, 5) (1, 100, 3) (1, 150, 5) (1, 150, 3) (1, 50, 5)


In [8]:
from simba.util import check_and_initialize_data

# SIMBa
# Standard parameters
parameters['ms_horizon'] = None # No multiple shooting
parameters['base_lambda'] = 1

# Tunable parameters
parameters['learning_rate'] = 0.001
parameters['grad_clip'] = 100
parameters['train_loss'] = F.mse_loss
parameters['val_loss'] = F.mse_loss
parameters['dropout'] = 0
parameters['device'] = 'cpu'

parameters['batch_size'] = 128
parameters['horizon'] = None        # Prediction horizon of SIMBa
parameters['stride'] = 1          # Lag between two time steps to start predicting from
parameters['horizon_val'] = None  # None means entire trajectories
parameters['stride_val'] = 1

# Identify the state only
parameters['id_D'] = True
parameters['input_output'] = True
parameters['learn_x0'] = True

# Enforce stability
parameters['stable_A'] = True
parameters['LMI_A'] = True

parameters['delta'] = None

# Evaluate classical sysID baselines
baselines_to_use['parsim_s'] = False # Fails for some reason?
baselines_to_use['parsim_p'] = False # Fails for some reason?

x0 = x0_val = x0_test = np.zeros((1,1,2))
U, U_val, U_test, X, X_val, X_test, Y, Y_val, Y_test, x0, x0_val, x0_test = check_and_initialize_data(U, U_val, U_test, X, X_val, X_test, Y, Y_val, Y_test, x0, x0_val, x0_test,
                                                                                                            verbose=parameters['verbose'], autonomous=parameters['autonomous'], 
                                                                                                            input_output=parameters['input_output'], device=parameters['device'])

In [9]:
nxs = [2, 3, 4, 5, 6, 7, 8, 10, 12, 15, 20]
for nx in nxs:
    x0 = x0_val = x0_test = torch.zeros((1,1,nx))
    names, baselines, times, train_ids, validation_ids, test_ids = identify_baselines(nx=nx, U=U, U_val=U_val, U_test=U_test, Y=Y, Y_val=Y_val, Y_test=Y_test,
                                                                            x0=x0, x0_val=x0_val, x0_test=x0_test, dt=dt,
                                                                            parameters=parameters, baselines_to_use=baselines_to_use)

    name = f'SIMBa_{nx}'
    simba = Simba(nx=nx, nu=nu, ny=ny, parameters=parameters)
    simba.fit(U, U_val=U_val, U_test=U_test, X=X, X_val=X_val, X_test=X_test, Y=Y, Y_val=Y_val, Y_test=Y_test, x0=x0, x0_val=x0_val, x0_test=x0_test, baselines_to_use=baselines_to_use)
    simba.save(directory=directory, save_name=name)

    names, times, train_ids, validation_ids, test_ids = eval_simba(simba, name, names, times, train_ids, validation_ids, test_ids,
                                                        U, U_val, U_test, X, X_val, X_test, Y, Y_val, Y_test, x0, x0_val, x0_test)
    save_results(directory=directory, save_name=f'Results_{nx}', names=names, times=times, train_ids=train_ids, 
                validation_ids=validation_ids, test_ids=test_ids, data=data)
    
    print_all_perf(names, times, train_ids, validation_ids, test_ids, Y, Y_val, Y_test)



Initilization starts, fitting A!
Epoch	Fitting loss
1	7.48E-01
50000	2.63E-05
100000	3.22E-06
150000	8.64E-07
Total initialization time:	01'07"
Best loss at epoch 66216:	2.55E-08

SIPPY-PARSIM-K performance (Train and validation are only measured on the
first trajectory if there are several for now):
Train loss	Val loss	Test loss
4.17E-02	7.89E-02	5.50E-01

Training of SIMBa starts!
Training data shape:	(1, 100, *)
Validation data shape:	(1, 150, *)
Test data shape:	(1, 50, *)

Epoch	Train loss	Val loss	Test loss
1	4.17E-02	9.59E+00	3.10E+00
1000	3.04E-01	2.57E-01	6.93E-01
2000	3.08E-01	2.61E-01	5.59E-01
3000	2.61E-01	1.98E-01	5.19E-01
4000	2.25E-01	1.85E-01	5.84E-01
5000	2.48E-01	2.14E-01	5.22E-01
6000	2.07E-01	1.62E-01	5.04E-01
7000	1.83E-01	1.46E-01	5.76E-01
8000	2.03E-01	1.87E-01	5.22E-01
9000	1.67E-01	1.45E-01	5.12E-01
10000	1.54E-01	1.28E-01	5.84E-01

Average time per 100 epochs:	02"
Total training time:		04'14"

Best model performance:
65	2.31E-01	9.19E-02	5.49E-01

Method		Tim